<a href="https://colab.research.google.com/github/Sonmale25/ML_mini_projects/blob/main/Fake_News_Classifier_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle competitions download -c fake-news

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 433, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [2]:
import numpy as np
import pandas as pd
df=pd.read_csv('/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [4]:
#Since it is text data and not regressive numerical data, we will drop NaN values
df=df.dropna()

In [5]:
df.shape
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
X=df.drop('label',axis=1)

In [7]:
y=df['label']

In [8]:
 import tensorflow as tf
 from tensorflow.keras.layers import Embedding
 from tensorflow.keras.preprocessing.sequence import pad_sequences
 from tensorflow.keras.models import Sequential
 from tensorflow.keras.preprocessing.text import one_hot
 from tensorflow.keras.layers import LSTM
 from tensorflow.keras.layers import Dense

In [9]:
voc_size=5000

**One Hot Rerpresentation**

In [14]:
messages=X.copy()
messages['title'][1]
messages.reset_index(inplace=True)

In [11]:
import nltk
import re
from nltk.corpus import stopwords

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# Dataset preprocessing
from nltk.stem.porter import PorterStemmer # For stemming
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review=review.lower()
  review=review.split() # Each title will be split into words
  # Stem and store only those words which aren't stopwords
  review=[ps.stem(words) for words in review if not words in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [ ]:
# Alternative:
"""
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]).lower().split()

    # Remove stopwords using filter
    filtered_words = filter(lambda word: word not in stopwords.words('english'), review)

    # Apply stemming using map
    stemmed_words = map(ps.stem, filtered_words)

    # Join and append
    corpus.append(' '.join(stemmed_words))"""

In [16]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [17]:
#Corpus is a list of list of sentences
#Each sentence will be passed to the encoder which will ec=ncode the words
# and store the word encodings in list of lists as per the sentences
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[1075, 3906, 4804, 127, 4225, 4434, 2168, 2544, 1246, 1058],
 [4787, 1884, 4709, 44, 4774, 4653, 4567],
 [1421, 931, 3777, 765],
 [779, 1534, 1614, 3917, 2651, 4722],
 [4538, 4774, 627, 1607, 3293, 3159, 4774, 3838, 1892, 3206],
 [4503,
  2118,
  3838,
  4992,
  1221,
  2346,
  492,
  3775,
  436,
  2679,
  2271,
  1965,
  4277,
  999,
  4567],
 [584, 3668, 4321, 4040, 272, 1278, 3455, 2819, 1512, 4138, 3430],
 [4052, 3731, 3713, 416, 4678, 247, 2346, 2128, 1512, 4138, 3430],
 [214, 4199, 2425, 3674, 3669, 3845, 956, 2858, 2346, 539],
 [3873, 1891, 4411, 2102, 2679, 2296, 3163, 3789],
 [4949, 485, 3208, 2533, 4797, 2760, 3582, 2510, 2663, 1473, 2834],
 [3917, 1680, 4225, 3845, 2346, 4678],
 [3770, 1073, 1328, 1164, 4563, 561, 3585, 3192, 2631],
 [3450, 3615, 707, 1368, 2957, 251, 624, 1512, 4138, 3430],
 [2581, 4647, 4946, 238, 4111, 1512, 4138, 3430],
 [4610, 1811, 1885, 1056, 4041, 2194, 1223, 4787, 3249, 638],
 [1226, 4539, 1884],
 [1573, 3414, 2690, 483, 2346, 3198, 2362, 4567],
 

In [ ]:
"""
from tensorflow.keras.preprocessing.text import one_hot

corpus = [
    "hello world",
    "good morning",
    "machine learning is fun"
]
voc_size = 100

one_hot_repr = [one_hot(sentence, voc_size) for sentence in corpus]
print(one_hot_repr)"""

# output:[[22, 54], [45, 78], [12, 34, 90, 67]]


In [18]:

onehot_repr[1]

[4787, 1884, 4709, 44, 4774, 4653, 4567]

In [19]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [20]:
sent_len=20 # max length for padding
padded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)
print(padded_docs)

[[   0    0    0 ... 2544 1246 1058]
 [   0    0    0 ... 4774 4653 4567]
 [   0    0    0 ...  931 3777  765]
 ...
 [   0    0    0 ... 1512 4138 3430]
 [   0    0    0 ... 4434 3979 1228]
 [   0    0    0 ... 3563 1478 2832]]


In [23]:
#Creating model
model=Sequential()
model.add(Embedding(voc_size,40,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [24]:
X_Final=np.array(padded_docs)
y_final=np.array(y)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X_Final,y_final, test_size=0.33, random_state=42)

**MODEL TRAINING**

In [26]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 48ms/step - accuracy: 0.7789 - loss: 0.4253 - val_accuracy: 0.9158 - val_loss: 0.1888
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9460 - loss: 0.1463 - val_accuracy: 0.9128 - val_loss: 0.2051
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.9695 - loss: 0.0922 - val_accuracy: 0.9137 - val_loss: 0.2302
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.9801 - loss: 0.0592 - val_accuracy: 0.9153 - val_loss: 0.2547
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.9842 - loss: 0.0467 - val_accuracy: 0.9109 - val_loss: 0.2845
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.9901 - loss: 0.0334 - val_accuracy: 0.9109 - val_loss: 0.3387
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.9939 - loss: 0.0204 - val_accuracy: 0.8954 - val_loss: 0.3881
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.9944 - loss: 0.0209 - val

In [27]:
y_pred=np.where(model.predict(X_test)>0.5,1,0)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step


In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3120,  299],
       [ 255, 2361]])

In [29]:
 from sklearn.metrics import accuracy_score
 accuracy_score(y_test,y_pred)

0.9082021541010771

In [32]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features))
model2.add(Dropout(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [33]:
model2.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 46ms/step - accuracy: 0.7715 - loss: 0.4556 - val_accuracy: 0.9070 - val_loss: 0.2135
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.9338 - loss: 0.1558 - val_accuracy: 0.9183 - val_loss: 0.1870
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.9586 - loss: 0.1147 - val_accuracy: 0.9173 - val_loss: 0.2008
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.9754 - loss: 0.0788 - val_accuracy: 0.9176 - val_loss: 0.2303
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 17s 76ms/step - accuracy: 0.9782 - loss: 0.0638 - val_accuracy: 0.9167 - val_loss: 0.2428
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 20s 72ms/step - accuracy: 0.9829 - loss: 0.0490 - val_accuracy: 0.9137 - val_loss: 0.2832
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - accuracy: 0.9870 - loss: 0.0379 - val_accuracy: 0.9127 - val_loss: 0.3573
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - accuracy: 0.9911 - loss: 0.0325 - va

In [34]:
y_pred2=np.where(model2.predict(X_test)>0.5,1,0)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [35]:
accuracy_score(y_test,y_pred2)

0.9116818558409279

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

